In [1]:
import abtem
import numpy as np

# Wave functions
The basic abTEM multislice algorithm works by propagating the `xy` part of the wave function forward through an electrostatic potential one thin slice at a time (see Appendix 1 for a full derivation).

The `Waves` object wraps a complex `numpy` array representing an arbitrary wave function on a grid (given by $\phi$ in Appendix 1). Here is how to define a plane wave on a $512 \times 512$ grid, a lateral extent of $10\times 10$ $\mathrm{Å}^2$ and an energy of 300 keV:

In [2]:
complex_array = np.ones((512, 512), dtype='complex64')

waves = abtem.Waves(complex_array, extent=10, energy=300e3)

The complex array can be accessed through the `array` property. 

In [3]:
waves.array is complex_array

True

The real-space sampling, or pixel size, is calculated by dividing the `extent` by the number of `grid points`. The properties related to the grid, i.e. the `extent`, grid points (`gpts`) and `sampling` can be accessed as:

In [4]:
waves.extent, waves.gpts, waves.sampling

((10.0, 10.0), (512, 512), (0.01953125, 0.01953125))

To facilitate the creation of wave functions, abTEM includes a number of wave function "builders". For example, the above is more easily accomplished by writing:

In [6]:
waves = abtem.PlaneWave(gpts=512, extent=10, energy=300e3, normalize=False)

waves = waves.build(lazy=False)

np.all(waves.array == 1. + 0.j)

True

## Real-space sampling

The real-space sampling is extremely important because it controls the accuracy of the simulation at high scattering angles. The sampling defines the maximum spatial frequency $k_{max}$ via the formula:

$$ k_{max} = \frac{1}{2p} \quad , $$

where $p$ is the real-space sampling distance. To counteract aliasing artifacts due to the periodicity assumption of a discrete Fourier transform, abTEM supresses spatial frequencies above 2 / 3 of the maximum scattering angle, further reducing the maximum effective scattering angle by a factor of 2 / 3. Hence the maximum scattering angle $\alpha_{max}$ is given by:

$$ \alpha_{max} = \frac{2}{3}\frac{\lambda}{2p} \quad , $$

where $\lambda$ is the relativistic electron wavelength. As an example, consider a case where we want to simulate 80 keV electron scattering up to angles of 200 mrads. Plugging these values into the above equation gives a sampling of $\sim0.052$ Å, i.e. we require at least 0.05 Å pixel size in order to reach a maximum scattering angle of 200 mrads. In practice, you should ensure that the simulation is converged with respect to pixel size.

The maximum scattering angles in the $x$- and $y$-direction of the wave functions object can obtained

In [8]:
print(f'Maximal simulated scattering angles = {waves.cutoff_angles[0]:.3f} mrad')

Maximal simulated scattering angles = 336.656 mrad


<div class= "alert alert-info">

**Note: Units**

abTEM uses the same unit conventions as ASE, as defined in the `ase.units` module. Thus, electron volts (eV), Ångström (Å), and atomic mass units are defined as 1.0. The electrostatic potential is given in (eV / e). Angles in abTEM are for convenience given in milliradians (mrad).

</div>

## Probe wave functions

In STEM, the wave function is a focused beam of electrons. The convention used in abTEM is a probe defined by

$$
    \phi(k) = A(k) \exp(-i \chi(k)) \quad ,
$$

where $A(k)$ is the condenser aperture function and $\chi(k)$ is the phase error. A detailed description of each term is given in the discussion of the Contrast Tranfer Function.

We create a probe with a sampling of $0.05 \ \mathrm{Å}$, an extent of $10\times 10$ $\mathrm{Å}^2$, an energy of $60 \ \mathrm{keV}$, a convergence semiangle of $30 \ \mathrm{mrad}$, a focal spread of of $30 \ \mathrm{Å}$ and defocus of $50 \ \mathrm{Å}$. 

In [18]:
probe = abtem.Probe(sampling = 0.05, 
                    extent = 10, 
                    energy = 100e3, 
                    semiangle_cutoff = 30,
                    defocus = 50)

We can build a batch of three probes at different positions.

In [19]:
positions = [(0, 0), (2.5, 2.5), (5, 5)]

waves = probe.build(positions, lazy=True)

waves.array

dask.array<_extract_measurement, shape=(3, 200, 200), dtype=complex64, chunksize=(3, 200, 200), chunktype=numpy.ndarray>

The last two dimensions of the `Waves` represent the spatial dimensions, we also call these the *base* dimensions of the `Waves`. Any dimension preceeding the base dimensions are called the *ensemble* dimensions, you can read about how abTEM uses the concept of ensembles [here]().

All ensemble dimensions may be sliced. However, slicing a base dimension will throw an error.

In [20]:
wave = waves[1]

wave.shape

(200, 200)

The wave function can be visualized using the `.show` method.

In [21]:
wave.show();

<div class= "alert alert-info">

**Note: Axes convention**

The axes convention for arrays representing a wave functions assigns the first axis to the $x$-axis and the second axis to the $y$-axis, and a larger array index implies a larger real-space coordinate. This is different from the convention often used for images. The correct way of displaying an array with `matplotplib` is given below.
    
```
import matplotlib.pyplot as plt
image = np.abs(waves.array[0]) ** 2
plt.imshow(image.T, extent=[0, waves.extent[0], 0, waves.extent[1]], origin='lower')
```     
  
</div>